# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

***

## Part III: Update Information and Mapping

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
from datetime import datetime

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


***
## Import Preprocessed Datasets

In [4]:
# Creates a DataFrame from reading a csv file 
Preps = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Preps_Unit_Cleaned.csv"))
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-74322,SAUCE|Fig Mayo,1.750,L,NaN,1750.0,ml
1,P-74613,SMD24|PREP|Fig Jam,650.000,g,NaN,650.0,g
2,P-74330,DRESSING|Garlic Tahini Dressin,750.000,ml,NaN,750.0,ml
3,P-71838,DRESSING|Goma Mayo,3.000,L,NaN,3000.0,ml
4,P-74391,DRESSING|Lemon Thyme,4.193,Kg,NaN,4193.0,g


In [5]:
# Creates a DataFrame from reading a csv file 
ghge_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "ghge_factors.csv"))
ghge_factors.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [6]:
# Creates a DataFrame from reading a csv file 
nitro_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "nitrogen_factors.csv"))
nitro_factors.head()

,Category ID,Food Category,g N lost/kg product
0,1,beef & buffalo meat,329.50
1,2,lamb/mutton & goat meat,231.15
2,3,pork (pig meat),132.80
3,4,"poultry (chicken, turkey)",116.80
4,5,butter,100.35


In [7]:
# Creates a DataFrame from reading a csv file 
water_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "water_factors.csv"))
water_factors.head()

,Category ID,Food Category,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,1,beef & buffalo meat,1677.200,61309.000
1,2,lamb/mutton & goat meat,461.200,258.900
2,3,pork (pig meat),1810.300,54242.700
3,4,"poultry (chicken, turkey)",370.300,333.500
4,5,butter,1010.176,50055.168


In [8]:
# Creates a DataFrame from reading a csv file 
land_factors = pd.read_csv(os.path.join(os.getcwd(), "data", "external", "land_factors.csv"))
land_factors.rename(columns={'km^2 land use/kg product': 'Land Use (m^2)'}, inplace=True)
land_factors['Land Use (m^2)'] *= 1000
land_factors.head()

,Category ID,Food Category,Land Use (m^2)
0,1,beef & buffalo meat,0.12645
1,2,lamb/mutton & goat meat,0.14320
2,3,pork (pig meat),0.02102
3,4,"poultry (chicken, turkey)",0.01151
4,5,butter,0.01395


In [9]:
# Load current Items List with assigned Emission Factors Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [10]:
new_items = pd.read_csv("data/mapping/new items added/New_Items_2024/New_Items_Added_2024-12-05.csv")
new_items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3969,4,CHICK FRYER HALAL 4 CUT=,1.00,Kg,1.0,Kg,POULTRY
1,I-3900,1,MEATBALL OVEN*BKD 1Z 160CT FZ=,2.00,bag,5.0,lb,MEAT
2,I-69779,24,PASTA ROTINI*35mm=,2.00,bag,5.0,lb,FOOD - GROCERY
3,I-3421,24,PASTA SPAGHETTI*20IN,9.07,Kg,1000.0,g,FOOD - GROCERY
4,I-2551,59,PASTA TORT*CHEESE=,5.00,Kg,1.0,Kg,FOOD - GROCERY
5,I-28657,24,PASTA|Cooked|Short|10kg,1.00,ea,10.0,Kg,PRODUCTION FOOD
6,I-3313,54,REGANO LEAF SHAKER TFC,12.00,ea,190.0,g,SPICES
7,I-68186,59,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY
8,I-2706,31,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY
9,I-56386,59,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY


In [11]:
item_nonstd = pd.read_csv("data/cleaning/Items_Nonstd.csv")
item_nonstd

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [12]:
preps_nonstd = pd.read_csv("data/cleaning/Preps_NonstdUom.csv")
preps_nonstd

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom


In [13]:
conv_updatecov = pd.read_csv("data/cleaning/update/Conv_UpdateConv.csv")
conv_updatecov

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398,g
1,I-28697,0.005181,1.0,ea,193.000,g
2,I-47441,0.005181,1.0,ea,193.000,g
3,I-1905,0.012500,1.0,CT,80.000,g
4,I-47440,0.005051,1.0,ea,198.000,g
...,...,...,...,...,...,...
655,I-71500,0.090909,1.0,each,11.000,g
656,I-74632,0.038095,10.0,ea,262.500,g
657,I-74633,0.038095,10.0,ea,262.500,g
658,I-74634,0.038095,10.0,ea,262.500,g


In [14]:
conv_updatecov.loc[conv_updatecov["ConversionId"] == "P-54581"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [15]:
Preps.loc[Preps["PrepId"] == "P-54581"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom


# Update Conversion_Added.csv

In [16]:
conversions_added = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv"))
conversions_added

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398,g
1,I-28697,0.005181,1.0,ea,193.000,g
2,I-47441,0.005181,1.0,ea,193.000,g
3,I-1905,0.012500,1.0,CT,80.000,g
4,I-47440,0.005051,1.0,ea,198.000,g
...,...,...,...,...,...,...
1227,I-71500,0.090909,1.0,each,11.000,g
1228,I-74632,0.038095,10.0,ea,262.500,g
1229,I-74633,0.038095,10.0,ea,262.500,g
1230,I-74634,0.038095,10.0,ea,262.500,g


In [17]:
# Concatenate the DataFrames
combined_df = pd.concat([conv_updatecov, conversions_added], ignore_index=True)
combined_df

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-67659,0.008818,1.0,each,113.398,g
1,I-28697,0.005181,1.0,ea,193.000,g
2,I-47441,0.005181,1.0,ea,193.000,g
3,I-1905,0.012500,1.0,CT,80.000,g
4,I-47440,0.005051,1.0,ea,198.000,g
...,...,...,...,...,...,...
1887,I-71500,0.090909,1.0,each,11.000,g
1888,I-74632,0.038095,10.0,ea,262.500,g
1889,I-74633,0.038095,10.0,ea,262.500,g
1890,I-74634,0.038095,10.0,ea,262.500,g


In [18]:
# Add the conv_updatecov file to the masterfile, Conversions_Added
path = os.path.join(os.getcwd(), "data", "cleaning", "Conversions_Added.csv")
combined_df.to_csv(path, index = False, header = True)

***
## Import Update Info

In [19]:
# Import list of prep that need convert uom to standard uom manually
Manual_PrepU = pd.read_csv(os.path.join(os.getcwd(), "data", "cleaning", "update", "Preps_UpdateUom.csv"))
# Here we can see that UOM examples are: each, ea, slice
Manual_PrepU.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-54697,LEMON|Wedge 1/8,8.0,each,PREP,84.0,g
1,P-35132,MARINATED|Lemon & Herb Chx,185.0,ea,PREP,24050.0,g
2,P-26234,BATCH|Roasted Garlic Bread,16.0,ea,PREP,1280.0,g
3,P-26170,GRILLED|NaanBread,1.0,ea,PREP,125.0,g
4,P-16305,YIELD|Smokie (1pc),1.0,ea,NaN,112.0,g


In [20]:
# The row of the DatFrame that contains the PrepId == P-54581
Manual_PrepU.loc[Manual_PrepU["PrepId"] == "P-54581"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
202,P-54581,SLICE|Multigrain Bread,22.0,slice,NaN,38.0,g


In [21]:
# Select the file path for new items list with category id
New_Items_Added = pd.read_csv("data/mapping/new items added/New_Items_2024/New_Items_Added_2024-12-05.csv")
New_Items_Added.tail(15)

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3969,4,CHICK FRYER HALAL 4 CUT=,1.00,Kg,1.0,Kg,POULTRY
1,I-3900,1,MEATBALL OVEN*BKD 1Z 160CT FZ=,2.00,bag,5.0,lb,MEAT
2,I-69779,24,PASTA ROTINI*35mm=,2.00,bag,5.0,lb,FOOD - GROCERY
3,I-3421,24,PASTA SPAGHETTI*20IN,9.07,Kg,1000.0,g,FOOD - GROCERY
4,I-2551,59,PASTA TORT*CHEESE=,5.00,Kg,1.0,Kg,FOOD - GROCERY
5,I-28657,24,PASTA|Cooked|Short|10kg,1.00,ea,10.0,Kg,PRODUCTION FOOD
6,I-3313,54,REGANO LEAF SHAKER TFC,12.00,ea,190.0,g,SPICES
7,I-68186,59,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY
8,I-2706,31,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY
9,I-56386,59,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY


In [22]:
# Import list of items that adjusted GHGe factor manually
Manual_Factor = pd.read_csv(os.path.join(os.getcwd(), "data", "mapping", "Manual_Adjust_Factors.csv"))
Manual_Factor.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU),Land Use (m^2)
0,I-52090,59.0,BURGER BEEF & MUSHROOM HALAL,1.0,cs,48.00,CT,MEAT,25.00894,200.86,1038.84,37961.2,0.0
1,I-45558,59.0,Prep-Vegan Parmesan,1000.0,g,1.00,g,PRODUCTION FOOD,3.85686,0.00,0.00,0.0,0.0
2,I-3352,59.0,MAYONNAISE PAIL TFC 4L,2.0,each,4.00,L,FOOD - GROCERY,3.55000,0.00,0.00,0.0,0.0
3,I-3223,59.0,COCONUT MILK 17/19% MILK FAT,6.0,LG CAN,2.84,L,FOOD - GROCERY,3.50000,0.00,1.00,1.0,0.0
4,I-2898,59.0,MUSTARD DIJON WINE FLEUR,6.0,jar,1.00,Kg,FOOD - GROCERY,3.32600,0.00,0.00,0.0,0.0


### Update Correct Uom for Preps

In [23]:
# Update prep list with manually adjusted uom
for index, row in Manual_PrepU.iterrows():
    PrepId = Manual_PrepU.loc[index, 'PrepId']
    qty = Manual_PrepU.loc[index, 'StdQty']
    uom = Manual_PrepU.loc[index, 'StdUom']
    Preps.loc[Preps['PrepId'] == PrepId, 'StdQty'] = qty
    Preps.loc[Preps['PrepId'] == PrepId, 'StdUom'] = uom

In [24]:
Preps.drop_duplicates(subset=['PrepId'], inplace=True,)

In [25]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-74322,SAUCE|Fig Mayo,1.750,L,NaN,1750.0,ml
1,P-74613,SMD24|PREP|Fig Jam,650.000,g,NaN,650.0,g
2,P-74330,DRESSING|Garlic Tahini Dressin,750.000,ml,NaN,750.0,ml
3,P-71838,DRESSING|Goma Mayo,3.000,L,NaN,3000.0,ml
4,P-74391,DRESSING|Lemon Thyme,4.193,Kg,NaN,4193.0,g


In [26]:
Preps.shape

(62, 7)

In [27]:
path = os.path.join(os.getcwd(), "data", "cleaning", "Preps_List_Cleaned.csv")
Preps.to_csv(path, index = False, header = True)

In [28]:
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [29]:
New_Items_Added.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-3969,4,CHICK FRYER HALAL 4 CUT=,1.00,Kg,1.0,Kg,POULTRY
1,I-3900,1,MEATBALL OVEN*BKD 1Z 160CT FZ=,2.00,bag,5.0,lb,MEAT
2,I-69779,24,PASTA ROTINI*35mm=,2.00,bag,5.0,lb,FOOD - GROCERY
3,I-3421,24,PASTA SPAGHETTI*20IN,9.07,Kg,1000.0,g,FOOD - GROCERY
4,I-2551,59,PASTA TORT*CHEESE=,5.00,Kg,1.0,Kg,FOOD - GROCERY


### Import List of New Items with Emission Factors Category ID Assigned

In [30]:
frames = [Items_Assigned, New_Items_Added]
Items_Assigned_Updated = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()
Items_Assigned_Updated.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN


In [31]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,NaN
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,NaN
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,NaN
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,NaN


In [32]:
Items_Assigned_Updated.shape

(3057, 9)

In [33]:
# Double brackets used to specify the column as a dataframe and not a series
# converting the 'CategoryID' column in the Items_Assigned_Updated dataframe to numeric data type using the pd.to_numeric()
# use the apply column to apply the coversion to the entire column
Items_Assigned_Updated[['CategoryID']] = Items_Assigned_Updated[['CategoryID']].apply(pd.to_numeric)

In [34]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,NaN
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,NaN
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,NaN
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,NaN


In [35]:
path = os.path.join(os.getcwd(), "data", "mapping", "Items_List_Assigned.csv")
Items_Assigned_Updated.to_csv(path, index = False, header = True)

## Mapping Items to Footprint Factors

In [36]:
Items_Assigned_Updated

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN
...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,NaN
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,NaN
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,NaN
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,NaN


In [37]:
# DEFINITION OF MAPPING: assigning certain attributes to data points based on criteria or predefined rules.

# Map GHG footprint factors
# merges two data frames Items_Assigned_Updated and ghge_factors on the columns CategoryID and Category ID, respectively. The 
# resulting data frame is stored in mapping.

# how='left' specifies that a left join is done between Items_Assigned_Updated and ghge_factors.
# left_on='CategoryID' specifies that the join condition for Items_Assigned_Updated should be based on the 'CategoryID' column.
# right_on='Category ID' specifies that the join condition for ghge_factors should be based on the 'Category ID' column.

mapping = pd.merge(Items_Assigned_Updated, ghge_factors.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')
# if row in mapping has CategoryId == NaN then it assigns the value of the column Active Total Supply Chain Emissions (kg CO2 / kg food)
# to zero
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0
# drops the columns Category ID and Food Category
display(mapping)
mapping = mapping.drop(columns=['Category ID', 'Food Category_x'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_x,Category ID,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,NaN,1.0,beef & buffalo meat,41.3463
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,NaN,1.0,beef & buffalo meat,41.3463
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,NaN,1.0,beef & buffalo meat,41.3463
...,...,...,...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,NaN,59.0,manually adjusted,0.0000
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,NaN,31.0,other fruits,0.4306
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,NaN,59.0,manually adjusted,0.0000
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,NaN,31.0,other fruits,0.4306


,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463
...,...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,manually adjusted,0.0000
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,other fruits,0.4306
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,manually adjusted,0.0000
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,other fruits,0.4306


In [38]:
# Map nitrogen footprint factors
mapping = pd.merge(mapping, nitro_factors.loc[:,['Category ID','Food Category','g N lost/kg product']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'g N lost/kg product'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.5
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.5
...,...,...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,manually adjusted,0.0000,0.0
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,other fruits,0.4306,2.7
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,manually adjusted,0.0000,0.0
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,other fruits,0.4306,2.7


In [39]:
# Map land footprint factors
mapping = pd.merge(mapping, land_factors.loc[:,['Category ID','Food Category','Land Use (m^2)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Land Use (m^2)'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5,0.12645
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5,0.12645
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5,0.12645
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.5,0.12645
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.5,0.12645
...,...,...,...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,manually adjusted,0.0000,0.0,0.00000
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,other fruits,0.4306,2.7,0.00071
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,manually adjusted,0.0000,0.0,0.00000
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,other fruits,0.4306,2.7,0.00071


In [40]:
# Map water footprint factors
# mapping: DataFrame that is used to assign sustainability-related factors (greenhouse gas emissions, nitrogen loss, and 
# water footprint) to food items based on their category and other attributes. 
mapping = pd.merge(mapping, water_factors.loc[:,['Category ID','Food Category','Freshwater Withdrawals (L/FU)', 'Stress-Weighted Water Use (L/FU)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Freshwater Withdrawals (L/FU)'] = 0
        mapping.loc[index,'Stress-Weighted Water Use (L/FU)'] = 0

mapping = mapping.drop(columns=['Category ID', 'Food Category'])
mapping.drop_duplicates(subset=["ItemId"], inplace=True)
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5,0.12645,1677.2,61309.0
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5,0.12645,1677.2,61309.0
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.5,0.12645,1677.2,61309.0
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.5,0.12645,1677.2,61309.0
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.5,0.12645,1677.2,61309.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,manually adjusted,0.0000,0.0,0.00000,0.0,0.0
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,other fruits,0.4306,2.7,0.00071,3.5,4.7
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,manually adjusted,0.0000,0.0,0.00000,0.0,0.0
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,other fruits,0.4306,2.7,0.00071,3.5,4.7


In [41]:
mapping[mapping["ItemId"] == "I-4524"]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
2006,I-4524,38.0,BEETS GOLDBC,5.0,lb,1.0,lb,PRODUCE,root vegetables,0.3062,7.9,0.00032,9.9,37.9


In [42]:
mapping["CategoryID"].isnull().sum()

np.int64(0)

### Manully Adjust Footprint Factor for Specific Items

In [43]:
# For Manual_Factor: dataframe with items that adjusted GHGe factor manually.
# It takes the id for Manual_Factor for the index being iterated and if it is equal to item id of mapping dataframe then it sets
# the values of the columns to the manually adjusted values
# Note: the values for the columns in mapping DataFrame is adjusted and not Manual_Factor DataFrame
for index, row in Manual_Factor.iterrows():
    itemId = Manual_Factor.loc[index, 'ItemId']
    ghge = Manual_Factor.loc[index, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
    nitro = Manual_Factor.loc[index, 'g N lost/kg product']
    water = Manual_Factor.loc[index, 'Freshwater Withdrawals (L/FU)']
    land = Manual_Factor.loc[index, 'Land Use (m^2)']
    str_water = Manual_Factor.loc[index, 'Stress-Weighted Water Use (L/FU)']
    mapping.loc[mapping['ItemId'] == itemId, 'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = ghge
    mapping.loc[mapping['ItemId'] == itemId, 'g N lost/kg product'] = nitro
    mapping.loc[mapping['ItemId'] == itemId, 'Freshwater Withdrawals (L/FU)'] = water
    mapping.loc[mapping['ItemId'] == itemId, 'Stress-Weighted Water Use (L/FU)'] = str_water
    mapping.loc[mapping['ItemId'] == itemId, 'Land Use (m^2)'] = land

In [44]:
mapping.drop_duplicates(subset = ['ItemId'], inplace=True)
mapping.dtypes

ItemId                                                     object
CategoryID                                                float64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Food Category_y                                            object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
g N lost/kg product                                       float64
Land Use (m^2)                                            float64
Freshwater Withdrawals (L/FU)                             float64
Stress-Weighted Water Use (L/FU)                          float64
dtype: object

In [45]:
mapping.shape

(3056, 14)

In [46]:
mapping

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.126450,1677.2,61309.00
1,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.126450,1677.2,61309.00
2,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT,beef & buffalo meat,41.3463,329.50,0.126450,1677.2,61309.00
3,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.126450,1677.2,61309.00
4,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.0,g,MEAT,beef & buffalo meat,41.3463,329.50,0.126450,1677.2,61309.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3052,I-68186,59.0,SAUCE ALFREDO*TRADITIONAL=,4.00,bag,2000.0,g,FOOD - GROCERY,manually adjusted,7.8000,52.38,0.000010,0.0,47765.90
3053,I-2706,31.0,SOUP BUTTERNUT*SQUASH NO MSG=,3.00,bag,4.0,lb,FOOD - GROCERY,other fruits,0.4306,2.70,0.000710,3.5,4.70
3054,I-56386,59.0,SPREAD GARLIC*NH ZTF=,8.00,Kg,1.0,Kg,FOOD - GROCERY,manually adjusted,9.6100,98.74,0.000014,0.0,54975.69
3055,I-4782,31.0,PEARS ANJOU(120 CT) US,5.00,lb,1.0,lb,PRODUCE,other fruits,0.4306,2.70,0.000710,3.5,4.70


In [47]:
ingredients = pd.read_csv("data/preprocessed/Ingredients_List.csv")
ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-3351,1.0,L,1.000,0.5714,P-74322
1,P-74613,750.0,g,1.000,428.5714,P-74322
2,I-3660,200.0,g,0.001,0.3077,P-74613
3,I-3692,45.0,ml,0.001,0.0692,P-74613
4,I-5983,250.0,ml,0.001,0.3846,P-74613
...,...,...,...,...,...,...
595,I-72653,1.0,ea,1.000,1.0000,R-74608
596,I-29692,1.0,ea,1.000,1.0000,R-74650
597,I-1109,1.0,CT,1.000,1.0000,R-74699
598,I-2306,32.0,g,1.000,1.0000,R-74699


In [48]:
map_list = mapping["ItemId"].unique()
absent_list = []

for item in ingredients["IngredientId"].unique():
    if item not in map_list and item.startswith("I-"):
        absent_list.append(item)

# absent_list contains the IngredientIds that are not present in mapping but are present in the ingredients DataFrame
print(absent_list)

[]


In [49]:
# Converts mapping DataFrame to the Mapping.csv file
path = os.path.join(os.getcwd(), "data", "mapping", "Mapping.csv")
mapping.to_csv(path, index = False, header = True)

In [50]:
mapping[mapping["ItemId"] == "I-19143"]

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Food Category_y,Active Total Supply Chain Emissions (kg CO2 / kg food),g N lost/kg product,Land Use (m^2),Freshwater Withdrawals (L/FU),Stress-Weighted Water Use (L/FU)
674,I-19143,21.0,CHIA SEEDS BLACK ORGANIC,5.0,Kg,1000.0,g,SPICES,other grains/cereals,1.4785,5.9,0.00454,677.075,10563.3
